<a target="_blank" href="https://colab.research.google.com/github/trchudley/GEOG2462/blob/main/Week_2_NDIs/1_Calculate_NDIs.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Calculate a normalised difference index in Google Earth Engine

## Log in to Google Earth Engine

Same as last week, we must log in to Google Earth Engine.
<!-- Note that we are importing an additional library this week - `geemap`'s 'chart' library. -->

In [1]:
import ee
import geemap
# import geemap.chart as chart

ee.Authenticate()  # Trigger the authentication flow.
ee.Initialize(project='ee-trchudley')    # Change to your own default project name.

## Search for a scene

Again, as last week, we will outline some search parameters. This week, we will look at Quelcayya Ice Cap (QIC), in Peru:

In [3]:

# Location - editable
latitude = -13.922           # Degrees of latitude
longitude = -70.821          # Degrees of longitude
size = 15000                 # Size of AOI, in metres
location_name = 'quelcayya'  # recognisable name, to create a useful file name

# Dates - editable
date_start = '2023-05-01'
date_end = '2023-09-30'

# Set up location geometry
point = ee.Geometry.Point(longitude, latitude)  # Create a point
region = point.buffer(size/2).bounds()  # Buffer the point to a 2D shape


Let's check we've got a good area. I had to increase the `size` parameter from 10 km to 15 km relative to last week's example to properly capture the full extent of the ice cap. You can explore increasing this as much as you like. The 'swath width' (width of the scene) of Landsat is 185 km, so we can go much larger. However, I wouldn't recommend it -

In [4]:
Map = geemap.Map()  # Create an empty Map
Map.addLayer(region, {}, "Search Region")  # Add our AOI
Map.centerObject(region, zoom=12)  # Centre our map on the region of interest
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

As with last week, we can find the least cloudy image in our search window:

In [5]:
# Get Landsat 8 image collection
landsat8_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")

# Filter to desired region and date bounds
landsat8_collection = landsat8_collection.filterBounds(region)
landsat8_collection = landsat8_collection.filterDate(date_start, date_end)

# Get the least cloudy image in the collection, and clip it to our search region
image = landsat8_collection.sort('CLOUD_COVER').first()
image = image.clip(region)

image

And below, we visualise it to make sure it's good quality. Note I have increased the `max_reflectance` value to 0.8, so that we can see features of the snow and ice. This has the side effect of making the bare ground look much darker.

In [7]:

Map = geemap.Map()  # Create empty map

max_reflectance = 0.80 # Set the upper limit of reflectance to visualise.
                       # Play with this value (between 0-1) to see what it
                       # does. It will need to be higher for snowy/icy
                       # scenes

visParams = {'bands': ['B4', 'B3', 'B2'], 'max': max_reflectance}
Map.addLayer(image, visParams, 'Colour Composite Image')

Map.centerObject(region, zoom=12)
Map

Map(center=[-13.921983704624434, -70.82089282499103], controls=(WidgetControl(options=['position', 'transparen…

# Calculating a normalised difference index

Recall from the lecture slides last week that a normalised difference index takes the form

$$NDI = \frac{r_{high} - r_{low}}{r_{high} + r_{low}}$$

Where $r_{high}$ is a band that has characteristically high reflectance for our surface of interest, and $r_{low}$ has characteristically low reflectance. From this principle, there are a number of useful normalised difference indices we can use (and more I'm sure you can find):

| Name | Equation |
| ---- | -------- |
| Normalised Difference Vegetation Index | $$NDVI = \frac{NIR_{B5} - Red_{B4}}{NIR_{B5} + Red_{B4}}$$ |
| Normalised Difference Snow Index | $$NDSI = \frac{Green_{B3} - SWIR_{B6}}{Green_{B3} + SWIR_{B6}}$$ |
| Normalised Difference Water Index | $$NDWI = \frac{Green_{B3} - NIR_{B5}}{Green_{B3} + NIR_{B5}}$$ |
| Normalised Difference Built-up Index | $$NDBI = \frac{SWIR_{B6} - NIR_{B5}}{SWIR_{B6} + NIR_{B5}}$$ |
| Normalised Difference Snow Index (ice) | $$NDWI_{ice} = \frac{Red_{B4} - Blue_{B2}}{Red_{B4} + Blue_{B2}}$$ |
| Normalised Burn Ratio | $$NBR = \frac{NIR_{B05} - SWIR_{B6}}{NIR_{B5} + SWIR_{B6}}$$ |

I have included the relevant Landsat 8/9 bands as subscript in the equations - note this won't be the same if you start using, e.g., Landsat 1-7 or Sentinel-2 in the future.

Google Earth Engine has a [fantastically useful function](https://developers.google.com/earth-engine/apidocs/ee-image-normalizeddifference) to help us simplify this: `ee.Image.normalizedDifference(['r_high', 'r_low'])` (note the American spelling of 'normalized'). This takes the name of two bands - $r_{high}$ and $r_{low}$ - and produces an NDI product.

All we have to do is make sure we know the correct names of the bands, which are as follows (they are also listed, with detailed information, [here](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_TOA#bands):

In [8]:
image.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'QA_PIXEL',
 'QA_RADSAT',
 'SAA',
 'SZA',
 'VAA',
 'VZA']

As we're currently looking at an ice cap, it makes sense to calculate NDSI for now. Consulting the table above, this means using the green band (band 3) and the short-wave infrared (SWIR) band (band 6). Let's do this now:

In [9]:

type_of_ndi = 'NDSI'  # type of NDI calculated, for filename purposes
r_high = 'B3'  # relevant band name for r_high
r_low = 'B6'   # relevant band name for r_low

ndi = image.normalizedDifference([r_high, r_low]).rename(type_of_ndi)

ndi

Now, we can visualise the product. Note that I have visualised both the NDSI and the colour image - click the 🔧 symbol in the top right, and then the ☰ button, to open a menu where you can toggle the NDWI on an off to compare the two.

In [ ]:

Map = geemap.Map() # Create empty map

# Display colour image
max_reflectance = 0.80
visParams = {'bands': ['B4', 'B3', 'B2'], 'max': max_reflectance}
Map.addLayer(image, visParams, 'True Colour')

# Display NDI
visParams = {'bands': [type_of_ndi], 'min': -1, 'max': 1, 'palette': ['red', 'white', 'blue']}
Map.addLayer(ndi, visParams, type_of_ndi)

Map.centerObject(region)
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Have a zoom around and see what you think. Overall, this is a good product - there's clear boundaries and not much confusion over shaded areas, etc. However, it's clear that the NDSI is also picking up lakes/water as ice. This is an established weakness of NDSI.

> **Task:**
>
> 1. Go back and calculate NDWI instead of NDSI. Is NDWI better at differentiating water and ice?
>
> 2. Based on the bands used, why do you think this is?
>
> 3. Have an explore of other NDIs as well (e.g. NDVI).

After you have explored this, return to calculating the NDSI and ensure the cells have run so that the `ndi` variable is storing NDSI.

# Thresholding

In [11]:
import geemap.chart as chart

my_sample = ndi.sample(region, 5000)

chart.feature_histogram(my_sample, type_of_ndi)

In [ ]:

histogramDictionary = ndi.reduceRegion(**{
  'reducer': ee.Reducer.histogram(30),
  # 'geometry': region.geometry(),
  'scale': 30,
  'maxPixels': 1e19
})

histogramDictionary


In [ ]:
chart.feature_histogram(histogramDictionary, type_of_ndi)

Exception: features must be an ee.FeatureCollection